In [1]:
import functools
from cij.util import convert_unit

class ResultsWriter:

    def __init__(self):
        self.registry = {}
        self.it = None
    
    def write_value_to_file(self, fname, unit_from, unit_to, value):
        value = convert_unit(unit_from, unit_to, value)
        print(value)
    
    def add_new_value_writer(self, keywords, f, unit_from, unit_to):
        print("Adding", keywords)
        for keyword in keywords:
            self.registry[keyword] = lambda: self.write_value_to_file("", unit_from, unit_to, f(self.it))
            
    def update_parent(self, it):
        self.it = it

    def register_value(self, keywords, unit_from, unit_to):
        print("Outside", keywords)
        def decorator(f):
#             print(args)
            self.add_new_value_writer(keywords, f, unit_from, unit_to)
            @functools.wraps(f)
            def wrapper(*args, **kwargs):
                return f(it, *args, **kwargs)
            return wrapper
        return decorator


    # def write_dict(self, f):
    #     @functools.wraps
    #     def wrapper(keywords, unit_from, unit_to):
    #         self.add_new_dict_writer(keywords, f, unit_from, unit_to)
    #         return f
    #     return wrapper
    
writer = ResultsWriter()

In [2]:
@writer.register_value(["haha"], "m", "km")
def test():
    return 5

Outside ['haha']
Adding ['haha']


In [3]:

# test()

In [4]:
# writer.registry["haha"]()

In [5]:
from lazy_property import LazyProperty

class TestClass:

    @writer.register_value(["height"], "m", "km")
    def height(self):
        return 5
    
    @writer.register_value(["width"], "m", "km")
    def width(self):
        return 7

Outside ['height']
Adding ['height']
Outside ['width']
Adding ['width']


In [6]:
s = TestClass()
writer.update_parent(s)

# print(s.height())
# print(s.height())

# print(s.width())
# print(s.width())

In [7]:
print(writer.registry)
writer.registry["width"]()
writer.registry["height"]()

{'haha': <function ResultsWriter.add_new_value_writer.<locals>.<lambda> at 0x124fb3840>, 'height': <function ResultsWriter.add_new_value_writer.<locals>.<lambda> at 0x124fb3ea0>, 'width': <function ResultsWriter.add_new_value_writer.<locals>.<lambda> at 0x125076510>}
0.007
0.005
